In [1]:
import os 
NUM_CPU = len(os.sched_getaffinity(0))
print(f'Number of CPUs: {NUM_CPU}')
NUM_THREADS = 1 
os.environ["OMP_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["MKL_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["OPENBIAS_NUM_THREADS"] = str(NUM_THREADS)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"]  = str(NUM_THREADS)
NUM_PROCESS = int(NUM_CPU // NUM_THREADS) 
print(f'Number of PROCESSs: {NUM_PROCESS}')

import warnings
warnings.filterwarnings('ignore')
import re
import numpy as np
np.set_printoptions(suppress=True)
import time
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from multiprocessing import Pool
from utils import *

Number of CPUs: 12


In [3]:
'''
GD
'''
def gd_multilogistic_opt_ic(x,y,K0_pval,baseclass, alpha): 

    ss,ncov = x.shape  
    K = len(np.unique(y))    
    y = get_onehot_org(y,baseclass) 
    dist=1.0; niter=0
    beta0 = np.zeros([ncov*(K-1),1]) 
    alpha0 = np.log((1/K0_pval-1)/(K-1))
    beta0[np.arange(K-1)*ncov] = alpha0
    while (dist>1.0e-5) & (niter<1500):
        niter=niter+1
        beta0mat = (beta0.reshape([(K-1),ncov]).T)*1.0
        link_mu = x@beta0mat  ## N*（K-1）
        prob = np.exp(link_mu);prob=prob/(1+np.sum(prob,axis = 1,keepdims=True)) 

        resid = y-prob
        D1=((x.T@resid/ss).T).reshape([-1,1]) # The first-order derivative 
        if niter == 200: alpha = 1 
        beta1 = beta0 + alpha * D1
        assert beta1.shape==(ncov*(K-1),1),'shape is wrong'
        dist = np.mean(np.abs(beta1-beta0))
        beta0 = beta1
        print(f'niter: {niter}, dist: {np.round(dist,6)}')
    return beta0.reshape([(K-1),ncov]).T, dist, niter  

'''
PMLE
'''
def mle_logistic_cpu_ic(k): 
    # Data generation
    idx_k = np.where(Y==k)[0] 
    idx_k = np.concatenate((idx_0,idx_k)) 
    x = X[idx_k]; y = Y[idx_k]
    # Optimization
    ss,ncov = x.shape  
    y = y.reshape(ss,1)
    y = 1*(y!=0) 
    dist=1.0; niter=0
    beta0 = np.zeros([ncov,1])
    alpha0 = np.log((1/K0_pval-1)/(K-1))
    beta0[0] = alpha0
    
    while (dist>1.0e-5) & (niter<100):
        niter=niter+1
        link_mu = x@beta0  
        prob = np.exp(link_mu);prob=prob/(1+prob)
        D1=x.T@(y-prob)/ss  # The first-order derivative
        weight = np.sqrt(prob*(1-prob))
        wx = weight*x 
        del weight
        D2 = wx.T@wx/ss+1.0e-3*np.eye(ncov)  # The second-order derivative
        del wx 
        step = np.linalg.inv(D2)@D1
        beta1 = beta0+step
        assert beta1.shape==(ncov,1),'shape is wrong'
        dist = np.mean(np.abs(beta1-beta0))
        beta0 = beta1
    return beta1.reshape([ncov,])

'''
SPMLE
'''
def mle_logistic_cpu_ic_unweighted(k):
    # Data generation
    idx_0_sub = np.random.choice(idx_0,int(idx_0.shape[0]*pi),replace=False) 
    idx_k = np.where(Y==k)[0] 
    idx_k = np.concatenate((idx_0_sub,idx_k)) 
    x = X[idx_k]; y = Y[idx_k]
    # Optimization
    ss,ncov = x.shape  
    y = y.reshape(ss,1)
    y = 1*(y!=0) 
    dist=1.0; niter=0
    beta0 = np.zeros([ncov,1])
    alpha0 = np.log(idx_k.shape[0]/idx_0_sub.shape[0]-1) 
    beta0[0] = alpha0

    while (dist>1.0e-5) & (niter<100):
        niter=niter+1
        link_mu = x@beta0  
        prob = np.exp(link_mu);prob=prob/(1+prob)
        D1=x.T@(y-prob)/ss  # The first-order derivative
        weight = np.sqrt(prob*(1-prob))
        wx = weight*x 
        del weight
        D2 = wx.T@wx/ss+1.0e-3*np.eye(ncov) # The second-order derivative
        del wx 
        step = np.linalg.inv(D2)@D1
        beta1 = beta0+step
        assert beta1.shape==(ncov,1),'shape is wrong'
        dist = np.mean(np.abs(beta1-beta0))
        beta0 = beta1
    return beta1.reshape([ncov,])

In [4]:
## Load data
traindata = np.load(f'/mnt/multi-class_simu/real data/Audi/data/512_traindata_8cls.npz')
testdata = np.load(f'/mnt/multi-class_simu/real data/Audi/data/512_testdata_8cls.npz')
X = traindata['X0']; Y = traindata['Y0']; Ns0 = traindata['Ns0']
X1 = testdata['X1']; Y1 = testdata['Y1']; Ns1 = testdata['Ns1']
X = np.concatenate((np.ones([X.shape[0],1]),X),1)
X1 = np.concatenate((np.ones([X1.shape[0],1]),X1),1)
Y = Y.astype(np.int16).reshape(-1)
Y1 = Y1.astype(np.int16).reshape(-1)
# Normalize the data.
X = feature_std(X); X1 = feature_std(X1) 

N = X.shape[0]  #  The total number of samples
p = X.shape[1]  # The feature dimension
K = 9           # The number of classes, including class 0
baseclass = 0
K0_pval = np.sum(Y == 0)/ Y.shape[0]
idx_0 = np.where(Y==0)[0]  
print(N, p, K )

44772 513 9


In [5]:
'''
GMLE + GD
'''
t0 = time.time();
alpha = 5 
beta_gmle_gd, _, _ = gd_multilogistic_opt_ic(X, Y, K0_pval, baseclass, alpha)
t1 = time.time(); 
print(f'GMLE(GD): {np.round(t1-t0,2)}s')

niter: 1, dist: 0.001226
niter: 2, dist: 0.001216
niter: 3, dist: 0.001207
niter: 4, dist: 0.001198
niter: 5, dist: 0.001191
niter: 6, dist: 0.001183
niter: 7, dist: 0.001176
niter: 8, dist: 0.001168
niter: 9, dist: 0.00116
niter: 10, dist: 0.001151
niter: 11, dist: 0.001142
niter: 12, dist: 0.001132
niter: 13, dist: 0.001122
niter: 14, dist: 0.001111
niter: 15, dist: 0.0011
niter: 16, dist: 0.001089
niter: 17, dist: 0.001077
niter: 18, dist: 0.001066
niter: 19, dist: 0.001054
niter: 20, dist: 0.001042
niter: 21, dist: 0.00103
niter: 22, dist: 0.001018
niter: 23, dist: 0.001006
niter: 24, dist: 0.000993
niter: 25, dist: 0.000981
niter: 26, dist: 0.000969
niter: 27, dist: 0.000957
niter: 28, dist: 0.000945
niter: 29, dist: 0.000933
niter: 30, dist: 0.000922
niter: 31, dist: 0.00091
niter: 32, dist: 0.000899
niter: 33, dist: 0.000887
niter: 34, dist: 0.000876
niter: 35, dist: 0.000865
niter: 36, dist: 0.000854
niter: 37, dist: 0.000844
niter: 38, dist: 0.000833
niter: 39, dist: 0.000823


niter: 316, dist: 5.1e-05
niter: 317, dist: 5.1e-05
niter: 318, dist: 5.1e-05
niter: 319, dist: 5.1e-05
niter: 320, dist: 5e-05
niter: 321, dist: 5e-05
niter: 322, dist: 5e-05
niter: 323, dist: 5e-05
niter: 324, dist: 5e-05
niter: 325, dist: 5e-05
niter: 326, dist: 5e-05
niter: 327, dist: 5e-05
niter: 328, dist: 5e-05
niter: 329, dist: 5e-05
niter: 330, dist: 5e-05
niter: 331, dist: 5e-05
niter: 332, dist: 5e-05
niter: 333, dist: 5e-05
niter: 334, dist: 5e-05
niter: 335, dist: 5e-05
niter: 336, dist: 5e-05
niter: 337, dist: 5e-05
niter: 338, dist: 5e-05
niter: 339, dist: 5e-05
niter: 340, dist: 5e-05
niter: 341, dist: 5e-05
niter: 342, dist: 5e-05
niter: 343, dist: 5e-05
niter: 344, dist: 5e-05
niter: 345, dist: 5e-05
niter: 346, dist: 5e-05
niter: 347, dist: 5e-05
niter: 348, dist: 5e-05
niter: 349, dist: 5e-05
niter: 350, dist: 5e-05
niter: 351, dist: 5e-05
niter: 352, dist: 4.9e-05
niter: 353, dist: 4.9e-05
niter: 354, dist: 4.9e-05
niter: 355, dist: 4.9e-05
niter: 356, dist: 4.9e-0

niter: 635, dist: 4.2e-05
niter: 636, dist: 4.2e-05
niter: 637, dist: 4.2e-05
niter: 638, dist: 4.2e-05
niter: 639, dist: 4.2e-05
niter: 640, dist: 4.2e-05
niter: 641, dist: 4.2e-05
niter: 642, dist: 4.2e-05
niter: 643, dist: 4.2e-05
niter: 644, dist: 4.2e-05
niter: 645, dist: 4.2e-05
niter: 646, dist: 4.2e-05
niter: 647, dist: 4.2e-05
niter: 648, dist: 4.2e-05
niter: 649, dist: 4.2e-05
niter: 650, dist: 4.2e-05
niter: 651, dist: 4.2e-05
niter: 652, dist: 4.2e-05
niter: 653, dist: 4.2e-05
niter: 654, dist: 4.2e-05
niter: 655, dist: 4.2e-05
niter: 656, dist: 4.2e-05
niter: 657, dist: 4.2e-05
niter: 658, dist: 4.2e-05
niter: 659, dist: 4.2e-05
niter: 660, dist: 4.2e-05
niter: 661, dist: 4.2e-05
niter: 662, dist: 4.2e-05
niter: 663, dist: 4.2e-05
niter: 664, dist: 4.2e-05
niter: 665, dist: 4.2e-05
niter: 666, dist: 4.2e-05
niter: 667, dist: 4.2e-05
niter: 668, dist: 4.2e-05
niter: 669, dist: 4.2e-05
niter: 670, dist: 4.2e-05
niter: 671, dist: 4.2e-05
niter: 672, dist: 4.2e-05
niter: 673, 

niter: 956, dist: 3.7e-05
niter: 957, dist: 3.7e-05
niter: 958, dist: 3.7e-05
niter: 959, dist: 3.7e-05
niter: 960, dist: 3.7e-05
niter: 961, dist: 3.7e-05
niter: 962, dist: 3.7e-05
niter: 963, dist: 3.7e-05
niter: 964, dist: 3.7e-05
niter: 965, dist: 3.7e-05
niter: 966, dist: 3.7e-05
niter: 967, dist: 3.7e-05
niter: 968, dist: 3.7e-05
niter: 969, dist: 3.6e-05
niter: 970, dist: 3.6e-05
niter: 971, dist: 3.6e-05
niter: 972, dist: 3.6e-05
niter: 973, dist: 3.6e-05
niter: 974, dist: 3.6e-05
niter: 975, dist: 3.6e-05
niter: 976, dist: 3.6e-05
niter: 977, dist: 3.6e-05
niter: 978, dist: 3.6e-05
niter: 979, dist: 3.6e-05
niter: 980, dist: 3.6e-05
niter: 981, dist: 3.6e-05
niter: 982, dist: 3.6e-05
niter: 983, dist: 3.6e-05
niter: 984, dist: 3.6e-05
niter: 985, dist: 3.6e-05
niter: 986, dist: 3.6e-05
niter: 987, dist: 3.6e-05
niter: 988, dist: 3.6e-05
niter: 989, dist: 3.6e-05
niter: 990, dist: 3.6e-05
niter: 991, dist: 3.6e-05
niter: 992, dist: 3.6e-05
niter: 993, dist: 3.6e-05
niter: 994, 

niter: 1262, dist: 3.3e-05
niter: 1263, dist: 3.3e-05
niter: 1264, dist: 3.3e-05
niter: 1265, dist: 3.3e-05
niter: 1266, dist: 3.3e-05
niter: 1267, dist: 3.3e-05
niter: 1268, dist: 3.3e-05
niter: 1269, dist: 3.3e-05
niter: 1270, dist: 3.3e-05
niter: 1271, dist: 3.3e-05
niter: 1272, dist: 3.3e-05
niter: 1273, dist: 3.3e-05
niter: 1274, dist: 3.3e-05
niter: 1275, dist: 3.3e-05
niter: 1276, dist: 3.3e-05
niter: 1277, dist: 3.3e-05
niter: 1278, dist: 3.3e-05
niter: 1279, dist: 3.3e-05
niter: 1280, dist: 3.3e-05
niter: 1281, dist: 3.3e-05
niter: 1282, dist: 3.3e-05
niter: 1283, dist: 3.3e-05
niter: 1284, dist: 3.3e-05
niter: 1285, dist: 3.3e-05
niter: 1286, dist: 3.3e-05
niter: 1287, dist: 3.3e-05
niter: 1288, dist: 3.3e-05
niter: 1289, dist: 3.3e-05
niter: 1290, dist: 3.3e-05
niter: 1291, dist: 3.2e-05
niter: 1292, dist: 3.2e-05
niter: 1293, dist: 3.2e-05
niter: 1294, dist: 3.2e-05
niter: 1295, dist: 3.2e-05
niter: 1296, dist: 3.2e-05
niter: 1297, dist: 3.2e-05
niter: 1298, dist: 3.2e-05
n

In [ ]:
'''
PMLE 
'''
K = 9
t0 = time.time();
with Pool(K-1) as pool:
    beta_pmle = np.array(pool.map(mle_logistic_cpu_ic, [k for k in range(1,K)])).T
t1 = time.time(); print(f'PMLE: {np.round(t1-t0,2)}s')

In [6]:
'''
SPMLE 
'''
np.random.seed(0)
pi = 0.25 # Sampling probability 
K = 9
t0 = time.time();
with Pool(K-1) as pool:
    beta_sub = np.array(pool.map(mle_logistic_cpu_ic_unweighted, [k for k in range(1,K)])).T

t1 = time.time(); 
print(f'SPMLE: {np.round(t1-t0,2)}s')

SPMLE: 16.45s


In [9]:
test_accw(X1, Y1, Ns0, beta_gmle_gd) # GMLE
test_accw(X1, Y1, Ns0, beta_pmle)    # PMLE
test_accw(X1, Y1, Ns0, beta_sub)     # SPMLE

9 cls acc = 0.836,auc = 0.997
9 cls acc = 0.835,auc = 0.999
9 cls acc = 0.824,auc = 0.999


In [27]:
# np.save('/mnt/multi-class_simu/real data/Audi/model_coef/512_beta_pmle_8cls.npy', beta_pmle)
# np.save('/mnt/multi-class_simu/real data/Audi/model_coef/512_beta_gmle_gd_8cls.npy', beta_gmle_gd)
# np.save('/mnt/multi-class_simu/real data/Audi/model_coef/512_beta_pmle_pi0.25_8cls.npy', beta_sub)